# Import Libraries

In [2]:
# Standard library imports
import os # allows access to OS-dependent functionalities
import sys # to manipulate different parts of the Python runtime environment
import math
import csv

# Libraries go get shop information
import requests  # for HTTP requests
from bs4 import BeautifulSoup # for HTML scrapping 

# Libraries to Generate random drivers information
import random
import string

import pandas as pd
import folium

# setting display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Get the current working directory
cwd = os.getcwd()

# Add the path of the utils directory to sys.path
utils_path = os.path.abspath(os.path.join(cwd, '..', 'utils'))
sys.path.append(utils_path)

#Preparing folder variables

main_folder = os.path.abspath(os.path.join(os.pardir))
data_folder = (main_folder + "/" +"data")
saved_models_folder = (data_folder + "/" + "saved_models")
raw_data = (data_folder + "/" + "_raw")
processed_data = (data_folder + "/" + "processed")
content_based_supervised_data = (main_folder + "/" + "processed" + "/" + "content_based_supervised")

# Generating random drivers information about location

In [13]:
import random
import string

madrid_lat_bounds = (40.416775, 40.465) # latitude bounds of Madrid
madrid_lon_bounds = (-3.703790, -3.678419) # longitude bounds of Madrid

drivers_location = []

#  generate 50 records with driver_id, lat, and lon values
for i in range(1, 51):
    driver_id = ''.join(random.choice('0123456789ABCDEF') for j in range(4)) # generate 4 character random id
    lat = round(random.uniform(madrid_lat_bounds[0], madrid_lat_bounds[1]), 6)
    lon = round(random.uniform(madrid_lon_bounds[0], madrid_lon_bounds[1]), 6)
    disponibility = random.choice([True, False]) # randomly assign True or False to disponibility
    drivers_location.append({'driver_id': driver_id, 'lat': lat, 'lon': lon, 'disponibility': disponibility})


# open a CSV file in write mode
with open(raw_data + "/" + 'drivers_location.csv', mode='w', newline='') as file:

    # create a writer object
    writer = csv.writer(file)

    # write the header row
    writer.writerow(['driver_id', 'lat', 'lon','disponibility'])

    # write the data rows
    for address in drivers_location:
        writer.writerow([address['driver_id'], address['lat'], address['lon'], address['disponibility']])

In [26]:
# read the CSV file into a DataFrame
df_drivers = pd.read_csv(raw_data + "/" + "drivers_location.csv")

In [27]:
df_drivers.head(50)

,driver_id,lat,lon,disponibility
0,E5F9,40.455104,-3.680650,True
1,5EDA,40.429065,-3.691279,True
2,C6B7,40.416776,-3.691992,True
3,2CD7,40.448905,-3.691812,True
4,815B,40.436809,-3.696615,True
5,29A2,40.418351,-3.688199,False
6,9ECF,40.452772,-3.687252,False
7,2965,40.418179,-3.684902,True
8,840F,40.425157,-3.695811,False
9,7E80,40.430515,-3.697936,True


In [16]:
df_orders = pd.read_csv(raw_data + "/" + "orders.csv")

In [17]:
df_orders.columns

Index(['order_id', 'driver_id', 'restaurant_id', 'customer_name',
       'customer_address', 'order_time', 'delivery_time', 'total_amount',
       'order_status'],
      dtype='object')

In [ ]:
df_orders["driver_id"].unique()

In [179]:
df_drivers["driver_id"].unique()

array(['B402', '7EF0', '4820', 'B13D', '6B4A', '1CCF', '0749', '4805',
       '0CB4', 'D43B', 'CCE8', '39A4', '5951', 'C891', 'F6B6', 'DFC5',
       'AF40', '25A4', 'EEB2', '0F28', '765A', '7EA4', 'A8BC', '50A2',
       'ACA2', '48D7', '9AC7', 'F764', '8E9E', '6113', '93DF', '5052',
       'D456', '32C6', '5EDD', 'F30B', '4976', '11E2', '8954', '97FC',
       '64C1', 'A8EB', 'F1A4', '7079', '196B', '702B', 'DBD4', '5B9B',
       '98C0', '4699'], dtype=object)

In [6]:
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")

In [7]:
df_shops.columns

Index(['id', 'name', 'category', 'formatted_address', 'latitude', 'longitude',
       'admin_region', 'locality', 'region'],
      dtype='object')

In [180]:
df_shops["id"].unique()

array(['4c321c9816adc9281693c19c', '593be9b516fa0435e49bbc2f',
       '4adcda36f964a5208b3b21e3', '4f6b0ec86b74877569ff1026',
       '4b953cc5f964a5200f9734e3', '5fa948caab654d07e1dbcc9b',
       '5004217de4b0524582e67aa0', '527d595411d2833c64f87ef7',
       '4adcda34f964a520073b21e3', '4adcda3bf964a5206a3d21e3',
       '50d41346e4b0f0bb6c158154', '4adcda39f964a520c13c21e3',
       '4bf852c05efe2d7fe7956a34', '4b71e2a6f964a52033622de3',
       '4adcda34f964a5202d3b21e3', '56745739498ea0db2b2e482a',
       '4b4e3e50f964a52097e626e3', '4cffc4d3ba1da1cddb118528',
       '595bc8e52bf9a971a2316f62', '59b4dd5da8eb604056db7ba6',
       '58ff9e7c6fd626300708b2ab', '4b45401af964a520150926e3',
       '4bab2fd1f964a52079983ae3', '4c9cdc768afca093b936f115',
       '58207d2b96b0d84ce7ab7b91', '4be6b9a9d4f7c9b6b4fd2620',
       '53ce8e61498e9707065bd96d', '59b3be7a9deb7d19c10eddc2',
       '4dfe6c56e4cd8e633feb8a26', '4b956ae6f964a52080a134e3',
       '4c0aaf20340720a1d1e48693', '5d065fab3f069d0023a

# Getting Shops information using Foursquare API

In [120]:
import requests
category_id = 13000
lat = 40.41517
lng = -3.71273
limit = 50

url = "https://api.foursquare.com/v3/places/search?ll={}%2C{}&radius={}&categories={}&limit={}".format(
            lat, 
            lng,
            radius,
            category_id,
            limit)

headers = {
    "accept": "application/json",
    "Authorization": "fsq3/yhR0cDdx5Y/JGfd12cmI90pn5vnosARY51e1sPopbU="
}

response = requests.get(url, headers=headers).json()

#Creating the necessary lists
places_list = []
# Extract the relevant information from the response
places = response['results']
for place in places:
    places_dict = {"id" : place["fsq_id"],
                   "name" : place["name"],
                   "category" : place["categories"][-1]["name"],
                   "formatted_address" : place["location"]["formatted_address"],
                   "latitude" : place["geocodes"]["main"]["latitude"],
                   "longitude" : place["geocodes"]["main"]["longitude"],
                   "admin_region" : place["location"]["admin_region"],
                   "locality" : place["location"]["locality"],
                   "region" : place["location"]["region"]

                    }
    places_list.append(places_dict) # Append the loop info to anime_list
places_list

[{'id': '4c321c9816adc9281693c19c',
  'name': 'La Mayor',
  'category': 'Tapas Restaurant',
  'formatted_address': 'Calle Mayor, 77 (C. Bailén), 28013 Madrid Madrid',
  'latitude': 40.415106,
  'longitude': -3.712051,
  'admin_region': 'Comunidad de Madrid',
  'locality': 'Madrid',
  'region': 'Madrid'},
 {'id': '593be9b516fa0435e49bbc2f',
  'name': 'Zúccaru',
  'category': 'Italian Restaurant',
  'formatted_address': 'Calle Vergara, 16, 28013 Madrid Madrid',
  'latitude': 40.417227,
  'longitude': -3.711617,
  'admin_region': 'Comunidad de Madrid',
  'locality': 'Madrid',
  'region': 'Madrid'},
 {'id': '4adcda36f964a5208b3b21e3',
  'name': 'Corral de la Morería',
  'category': 'Spanish Restaurant',
  'formatted_address': 'Calle Morería, 17, 28005 Madrid Madrid',
  'latitude': 40.412736,
  'longitude': -3.714198,
  'admin_region': 'Comunidad de Madrid',
  'locality': 'Madrid',
  'region': 'Madrid'},
 {'id': '4f6b0ec86b74877569ff1026',
  'name': 'La Rebelión de los Mandiles',
  'categor

In [127]:
# open a CSV file in write mode
with open(raw_data + "/" + 'shops_location.csv', mode='w', newline='') as file:

    # create a writer object
    writer = csv.writer(file)

    # write the header row
    writer.writerow(['id', 'name', 'category','formatted_address','latitude','longitude','admin_region','locality','region'])

    # write the data rows
    for place in places_list:
        writer.writerow([place['id'], place['name'], place['category'], place['formatted_address'], place['latitude'], place['longitude'], place['admin_region'], place['locality'], place['region']])

In [129]:
# read the CSV file into a DataFrame
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")
df_shops.head(2)

,id,name,category,formatted_address,latitude,longitude,admin_region,locality,region
0,4c321c9816adc9281693c19c,La Mayor,Tapas Restaurant,"Calle Mayor, 77 (C. Bailén), 28013 Madrid Madrid",40.415106,-3.712051,Comunidad de Madrid,Madrid,Madrid
1,593be9b516fa0435e49bbc2f,Zúccaru,Italian Restaurant,"Calle Vergara, 16, 28013 Madrid Madrid",40.417227,-3.711617,Comunidad de Madrid,Madrid,Madrid


# Creating orders_table

In [32]:
import pandas as pd
import numpy as np

names = ['Emma', 'Liam', 'Olivia', 'Noah', 'Ava', 'Ethan', 'Isabella', 
         'James', 'Sophia', 'Mason', 'Mia', 'Alexander', 'Charlotte', 
         'William', 'Amelia', 'Michael', 'Harper', 'Benjamiformulas.n', 'Evelyn', 
         'Elijah', 'Abigail', 'Daniel', 'Emily', 'Matthew', 'Ella', 'Aiden', 
         'Elizabeth', 'David', 'Camila', 'Joseph', 'Chloe', 'Jackson', 'Victoria', 
         'Samuel', 'Grace', 'Lucas', 'Avery', 'Caleb', 'Sofia', 'Owen', 'Aria', 
         'Logan', 'Scarlett', 'Gabriel', 'Madison', 'Julian', 'Lily', 'Isaac', 
         'Eleanor', 'Levi', 'Hannah']

addresses = [
    "Calle de Alcalá, 123, 28009 Madrid",
    "Calle de Atocha, 14, 28012 Madrid",
    "Calle de Serrano, 24, 28001 Madrid",
    "Calle de Gran Vía, 1, 28013 Madrid",
    "Calle de Goya, 12, 28001 Madrid",
    "Paseo de la Castellana, 200, 28046 Madrid",
    "Calle de Preciados, 1, 28013 Madrid",
    "Plaza Mayor, s/n, 28012 Madrid",
    "Calle de Fuencarral, 53, 28004 Madrid",
    "Calle de la Princesa, 53, 28008 Madrid",
    "Calle de Hortaleza, 100, 28004 Madrid",
    "Calle de Bravo Murillo, 377, 28020 Madrid",
    "Calle de Velázquez, 150, 28002 Madrid",
    "Calle de la Montera, 24, 28013 Madrid",
    "Calle de Orense, 70, 28020 Madrid",
    "Calle de Alfonso XII, 3, 28014 Madrid",
    "Paseo de la Habana, 24, 28036 Madrid",
    "Calle del Carmen, 2, 28013 Madrid",
    "Calle de Don Ramón de la Cruz, 26, 28001 Madrid",
    "Calle del Pez, 27, 28004 Madrid",
    "Calle de Pradillo, 22, 28002 Madrid",
    "Calle de Segovia, 17, 28005 Madrid",
    "Calle de Sainz de Baranda, 39, 28009 Madrid",
    "Calle de López de Hoyos, 327, 28043 Madrid",
    "Calle de la Victoria, 5, 28012 Madrid",
    "Calle de la Princesa, 3, 28008 Madrid",
    "Calle de los Madrazo, 11, 28014 Madrid",
    "Calle de San Bernardo, 103, 28015 Madrid",
    "Calle de San Bernardo, 38, 28015 Madrid",
    "Calle de María de Molina, 20, 28006 Madrid",
    "Calle de Ibiza, 38, 28009 Madrid",
    "Calle del Marqués de Urquijo, 4, 28008 Madrid",
    "Calle de Bailén, 9, 28013 Madrid",
    "Calle de José Ortega y Gasset, 40, 28006 Madrid",
    "Calle de Génova, 11, 28004 Madrid",
    "Calle de Zurbano, 26, 28010 Madrid",
    "Calle de la Reina, 15, 28004 Madrid",
    "Calle de Alcalá, 57, 28014 Madrid",
    "Calle de Toledo, 46, 28005 Madrid",
    "Calle de Arturo Soria, 126, 28043 Madrid",
    "Calle del Príncipe de Vergara, 211, 28002 Madrid",
    "Calle de Alcalá, 203, 28028 Madrid",
    "Calle de Marqués de Lema, 7, 28028 Madrid"]

# create a list of driver IDs and restaurant IDs
driver_ids = ['B402', '7EF0', '4820', 'B13D', '6B4A', '1CCF', '0749', '4805',
              '0CB4', 'D43B', 'CCE8', '39A4', '5951', 'C891', 'F6B6', 'DFC5',
              'AF40', '25A4', 'EEB2', '0F28', '765A', '7EA4', 'A8BC', '50A2',
              'ACA2', '48D7', '9AC7', 'F764', '8E9E', '6113', '93DF', '5052',
              'D456', '32C6', '5EDD', 'F30B', '4976', '11E2', '8954', '97FC',
              '64C1', 'A8EB', 'F1A4', '7079', '196B', '702B', 'DBD4', '5B9B',
              '98C0', '4699']

restaurant_ids = ['4c321c9816adc9281693c19c', '593be9b516fa0435e49bbc2f',
       '4adcda36f964a5208b3b21e3', '4f6b0ec86b74877569ff1026',
       '4b953cc5f964a5200f9734e3', '5fa948caab654d07e1dbcc9b',
       '5004217de4b0524582e67aa0', '527d595411d2833c64f87ef7',
       '4adcda34f964a520073b21e3', '4adcda3bf964a5206a3d21e3',
       '50d41346e4b0f0bb6c158154', '4adcda39f964a520c13c21e3',
       '4bf852c05efe2d7fe7956a34', '4b71e2a6f964a52033622de3',
       '4adcda34f964a5202d3b21e3', '56745739498ea0db2b2e482a',
       '4b4e3e50f964a52097e626e3', '4cffc4d3ba1da1cddb118528',
       '595bc8e52bf9a971a2316f62', '59b4dd5da8eb604056db7ba6',
       '58ff9e7c6fd626300708b2ab', '4b45401af964a520150926e3',
       '4bab2fd1f964a52079983ae3', '4c9cdc768afca093b936f115',
       '58207d2b96b0d84ce7ab7b91', '4be6b9a9d4f7c9b6b4fd2620',
       '53ce8e61498e9707065bd96d', '59b3be7a9deb7d19c10eddc2',
       '4dfe6c56e4cd8e633feb8a26', '4b956ae6f964a52080a134e3',
       '4c0aaf20340720a1d1e48693', '5d065fab3f069d0023aa21ee',
       '5564344c498ead26929c9919', '4b7db2a1f964a52018cf2fe3',
       '4b688933f964a520827e2be3', '52f501e3498e2eb1dc055fe0',
       '4bcefa7068f976b0cbcc6683', '4bb39f2b42959c740d35222c',
       '4bf825cb4a67c9289cde24cf', '534ae3cf498e8c4e374a2d37',
       '4bd30ee041b9ef3bda22ffe5', '54a8494b498edea25d0cc4c0',
       '5c7879eb8194fc002c9f0ca0', '4b65ac89f964a520fef82ae3',
       '4c5030048edf0f479e49d5ae', '4c9e3d6d542b224b3c0df29f',
       '5375142f498efb5d19f0705c', '4adcda3af964a520353d21e3',
       '4af1d477f964a52096e321e3', '4b6ee7d1f964a52038d02ce3']

# generate 20 random records
np.random.seed(123)
order_table = pd.DataFrame({
    'order_id': np.arange(100),
    'driver_id': np.random.choice(driver_ids, 100),
    'restaurant_id': np.random.choice(restaurant_ids, 100),
    'customer_name': np.random.choice(names, 100),
    'customer_address': np.random.choice(addresses, 100),
    'order_time': pd.date_range(start='2022-01-01', periods=100, freq='H'),
    'delivery_time': pd.date_range(start='2022-01-01 01:00', periods=100, freq='H'),
    'total_amount': np.random.uniform(10, 50, 100),
    'order_status': np.random.choice(['delivered'], 100)
})

# display the resulting dataframe
order_table.head()

,order_id,driver_id,restaurant_id,customer_name,customer_address,order_time,delivery_time,total_amount,order_status
0,0,702B,5004217de4b0524582e67aa0,Camila,"Calle de Atocha, 14, 28012 Madrid",2022-01-01 00:00:00,2022-01-01 01:00:00,47.864836,delivered
1,1,4820,4adcda3bf964a5206a3d21e3,Alexander,"Calle de Génova, 11, 28004 Madrid",2022-01-01 01:00:00,2022-01-01 02:00:00,46.529250,delivered
2,2,8E9E,4c9cdc768afca093b936f115,Mia,"Calle de Marqués de Lema, 7, 28028 Madrid",2022-01-01 02:00:00,2022-01-01 03:00:00,12.050196,delivered
3,3,5EDD,4adcda34f964a5202d3b21e3,Noah,"Calle de Don Ramón de la Cruz, 26, 28001 Madrid",2022-01-01 03:00:00,2022-01-01 04:00:00,25.212619,delivered
4,4,8954,4bf825cb4a67c9289cde24cf,Julian,"Calle de Arturo Soria, 126, 28043 Madrid",2022-01-01 04:00:00,2022-01-01 05:00:00,32.554262,delivered


In [33]:
# create the DataFrame
df = pd.DataFrame(order_table)
df.head(2)

,order_id,driver_id,restaurant_id,customer_name,customer_address,order_time,delivery_time,total_amount,order_status
0,0,702B,5004217de4b0524582e67aa0,Camila,"Calle de Atocha, 14, 28012 Madrid",2022-01-01 00:00:00,2022-01-01 01:00:00,47.864836,delivered
1,1,4820,4adcda3bf964a5206a3d21e3,Alexander,"Calle de Génova, 11, 28004 Madrid",2022-01-01 01:00:00,2022-01-01 02:00:00,46.529250,delivered


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          100 non-null    int64         
 1   driver_id         100 non-null    object        
 2   restaurant_id     100 non-null    object        
 3   customer_name     100 non-null    object        
 4   customer_address  100 non-null    object        
 5   order_time        100 non-null    datetime64[ns]
 6   delivery_time     100 non-null    datetime64[ns]
 7   total_amount      100 non-null    float64       
 8   order_status      100 non-null    object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 7.2+ KB


In [35]:
# Save the dataframe as CSV
df.to_csv(raw_data + "/" + 'orders.csv', index=False)


# Practizing location

In [161]:
# read the drivers_location CSV file into a DataFrame 
df_drivers = pd.read_csv(raw_data + "/" + "drivers_location.csv")
df_drivers.head(5)

,driver_id,lat,lon,disponibility
0,A8EB,40.291556,-3.629250,True
1,DFC5,40.339510,-3.741539,True
2,B13D,40.448745,-3.693041,False
3,ACA2,40.349852,-3.612646,True
4,32C6,40.300321,-3.716476,True


In [168]:
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")
df_shops.head()

,id,name,category,formatted_address,latitude,longitude,admin_region,locality,region
0,4c321c9816adc9281693c19c,La Mayor,Tapas Restaurant,"Calle Mayor, 77 (C. Bailén), 28013 Madrid Madrid",40.415106,-3.712051,Comunidad de Madrid,Madrid,Madrid
1,593be9b516fa0435e49bbc2f,Zúccaru,Italian Restaurant,"Calle Vergara, 16, 28013 Madrid Madrid",40.417227,-3.711617,Comunidad de Madrid,Madrid,Madrid
2,4adcda36f964a5208b3b21e3,Corral de la Morería,Spanish Restaurant,"Calle Morería, 17, 28005 Madrid Madrid",40.412736,-3.714198,Comunidad de Madrid,Madrid,Madrid
3,4f6b0ec86b74877569ff1026,La Rebelión de los Mandiles,Deli,"Calle Mayor, 88, 28013 Madrid Madrid",40.415218,-3.713306,Comunidad de Madrid,Madrid,Madrid
4,4b953cc5f964a5200f9734e3,Restaurante Taberneros,Spanish Restaurant,"Santiago, 9, 28013 Madrid Madrid",40.416388,-3.710516,Comunidad de Madrid,Madrid,Madrid


In [172]:
df_drivers = pd.read_csv(raw_data + "/" + "drivers_location.csv")
df_drivers.columns

Index(['driver_id', 'lat', 'lon', 'disponibility'], dtype='object')

In [173]:
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")
df_shops.columns

Index(['id', 'name', 'category', 'formatted_address', 'latitude', 'longitude',
       'admin_region', 'locality', 'region'],
      dtype='object')

In [174]:
import numpy as np
from math import radians, cos, sin, asin, sqrt

# read the drivers_location CSV file into a DataFrame 
df_drivers = pd.read_csv(raw_data + "/" + "drivers_location.csv")

# Convert a Pandas DataFrame df_drivers to a list of dictionaries
driver_dicts = df_drivers.to_dict(orient='records')

# read the shops_location CSV file into a DataFrame
df_shops = pd.read_csv(raw_data + "/" + "shops_location.csv")

# Convert a Pandas DataFrame df_shops to a list of dictionaries
shops_dicts = df_shops.to_dict(orient='records')

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def find_closest_driver(df_drivers, df_shops, shop_name):
    # Get the latitude and longitude of the specified shop
    shop_location = df_shops.loc[df_shops['name'] == shop_name, ['latitude', 'longitude']].values.flatten()

    # Calculate the distance between each driver's location and the shop location using the Haversine formula
    distances = np.array([haversine(row['lon'], row['lat'], shop_location[1], shop_location[0]) for _, row in df_drivers.iterrows()])

    # Add the distances as a new column in the drivers DataFrame
    df_drivers['distance'] = distances

    # Sort the drivers by their distances to the shop location
    df_drivers.sort_values('distance', inplace=True)

    # Get the ID of the closest available driver to the shop location
    closest_driver_id = df_drivers.loc[(df_drivers['disponibility'] == True) & (df_drivers['distance'] > 0), 'driver_id'].values[0]

    return closest_driver_id

In [177]:
# Ask the user to input a shop name
shop_name = input("Enter a shop name: ")
closest_driver_id = find_closest_driver(df_drivers, df_shops, shop_name)
print(f'The closest available driver to {shop_name} is {closest_driver_id}')

The closest available driver to La Mayor is B402


,order_id,driver_id,restaurant_id,customer_name,customer_address,order_time,delivery_time,total_amount,order_status
0,0,702B,4b71e2a6f964a52033622de3,Customer 0,Address 0,2022-01-01 00:00:00,2022-01-01 01:00:00,24.309157,delivered
1,1,4820,4b953cc5f964a5200f9734e3,Customer 1,Address 1,2022-01-01 01:00:00,2022-01-01 02:00:00,26.688398,delivered
2,2,8E9E,4c321c9816adc9281693c19c,Customer 2,Address 2,2022-01-01 02:00:00,2022-01-01 03:00:00,36.188852,pending
3,3,5EDD,4c321c9816adc9281693c19c,Customer 3,Address 3,2022-01-01 03:00:00,2022-01-01 04:00:00,24.952057,delivered
4,4,8954,4b953cc5f964a5200f9734e3,Customer 4,Address 4,2022-01-01 04:00:00,2022-01-01 05:00:00,19.380515,delivered
